In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

#del train,test
#train=pd.read_csv("../input/train.csv")
#test=pd.read_csv("../input/test.csv")
#del train

In [ ]:
train=pd.read_csv("../input/train.csv")[:3000]

In [ ]:
train['eeg_fp1'].rolling(256).kurt().plot()

In [ ]:
dtypes = {"crew": "int8",
          "experiment": "category",
          "time": "float32",
          "seat": "int8",
          "eeg_fp1": "float32",
          "eeg_f7": "float32",
          "eeg_f8": "float32",
          "eeg_t4": "float32",
          "eeg_t6": "float32",
          "eeg_t5": "float32",
          "eeg_t3": "float32",
          "eeg_fp2": "float32",
          "eeg_o1": "float32",
          "eeg_p3": "float32",
          "eeg_pz": "float32",
          "eeg_f3": "float32",
          "eeg_fz": "float32",
          "eeg_f4": "float32",
          "eeg_c4": "float32",
          "eeg_p4": "float32",
          "eeg_poz": "float32",
          "eeg_c3": "float32",
          "eeg_cz": "float32",
          "eeg_o2": "float32",
          "ecg": "float32",
          "r": "float32",
          "gsr": "float32",
          "event": "category",
         }

In [ ]:
def corranal(train,test):
    excluded_feats = ['Name', 'RescuerID', 'Description', 'PetID','AdoptionSpeed','label_description'] #['SK_ID_CURR']
    features = [f_ for f_ in train.columns if f_ not in excluded_feats]

    print('Number of features %d  ' % (len(features)),train.shape,train.AdoptionSpeed.shape) 
    #effect_sizes = cohen_effect_size(Xtrain[:len(ytrain)], ytrain)
    effect_sizes = cohen_effect_size(train[features],train.AdoptionSpeed.values)
    effect_sizes.reindex(effect_sizes.abs().sort_values(ascending=False).nlargest(50).index)[::-1].plot.barh(figsize=(6, 10));
    print('Features with the 30 largest effect sizes')
    significant_features = [f for f in features if np.abs(effect_sizes.loc[f]) > 0.1]
    print('Significant features %d: %s' % (len(significant_features), significant_features))
    return significant_features

In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn import preprocessing
svd = TruncatedSVD(n_components=300,n_iter=7, random_state=42)

def kodeer(dfdir):
    chunk=5000000
    colo=256
    traine=[]
    for start in range(0,5000000,chunk):
        mtrain=pd.read_csv(dfdir,dtype=dtypes)[start:start+chunk]
        leng=int( len(mtrain)/colo)-1

        traint=mtrain["eeg_fp1"][:leng*colo].values.reshape(leng,colo)
        if len(traint)>0:
            for ci in  [ "eeg_f7", "eeg_f8", "eeg_t4", "eeg_t6", "eeg_t5", "eeg_t3", "eeg_fp2", "eeg_o1", "eeg_p3", "eeg_pz", "eeg_f3", "eeg_fz", "eeg_f4", "eeg_c4", "eeg_p4", "eeg_poz", "eeg_c3", "eeg_cz", "eeg_o2", "ecg", "r", "gsr"]:
                trainm=mtrain[ci].rolling(256).kurt()
                trainm=trainm.dropna()[:leng*colo].values.reshape(leng,colo)
                #print(trainm.shape,traint.shape)
                traint=np.hstack((traint,trainm))
            

        traint= traint.reshape(-1,23,256,1)

        
        if 'event' in mtrain.columns:    
            le = preprocessing.LabelEncoder()
            Yle=le.fit_transform(mtrain['event'])
            Yle=Yle[:leng*colo].reshape(leng,colo).mean(axis=1) 
            Yle=[np.int(x) for x in Yle]        
        else:
            Yle='' 

        
    return traint,Yle

#test,_=kodeer("../input/test.csv")
train,Yle=kodeer("../input/train.csv")


In [ ]:
import keras

np.random.seed(2)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau



model = Sequential()
# The kernel filter matrix is applied on the whole image

model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (23,256,1)))

# The CNN can isolate features that are useful everywhere from these transformed
# images(feature maps)

# The Second important layer in CNN is the pooling(MaxPool2D) layer.
# This layer simply acts as downsampling filter. 
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2))) # we have to choose the area size-
# -pooled each time(pool_size) more the pooling dimension is high, more the
# downsampling is important

# Dropout is a regularization technique for neural network model.
# A simple way to prevent neural network from overfitting
model.add(Dropout(0.25))

# Combining convolutional and pooling layers, CNN are able to combine local
# features and learn more global features of the image
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
# 'relu' is the rectifier(activation function = max(0,x)) is used to add 
# non-linearity to the network
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

# The Flatten layer is used to convert the final feature maps into 
# a one single 1D Vector
model.add(Flatten())

# Two Fully-connected(Dense =  10, activation = softmax) layers
# the net outputs distribution of probability of each class
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(5, activation = "softmax"))

# In summary() we can understand the architechture we build and 
# we can understand about the totol number of parameters 
model.summary()

# total_params = (filter_height * filter_width * input_image_channels + 1) * number_of_filters
# Define the optimizer
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

# Compile the model
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [ ]:
# Set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.0001)
epochs = 2
batch_size = 86

In [ ]:
# With data augmentation to prevent overfitting

datagen = ImageDataGenerator(
        featurewise_center=True,  # set input mean to 0 over the dataset
        samplewise_center=True,  # set each sample mean to 0
        featurewise_std_normalization=True,  # divide inputs by std of the dataset
        samplewise_std_normalization=True,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        #rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        #height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(train)

In [ ]:
# Encode labels to one hot vectors (ex : 2 -> [0,0,1,0,0,0,0,0,0,0])
Y_train = to_categorical(Yle, num_classes = 5)


In [ ]:
# Fit the model
history = model.fit_generator(datagen.flow(train[:14000],Y_train[:14000], batch_size=batch_size),
                              epochs = epochs, validation_data = (train[14000:],Y_train[14000:]),
                              verbose = 2, steps_per_epoch=train.shape[0] // batch_size
                              , callbacks=[learning_rate_reduction])

In [ ]:
test,_=kodeer("../input/test.csv")

In [ ]:
test.shape

In [ ]:
# Predict the values from the validation dataset
Y_pred = model.predict(test)
Y_pred